In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torchvision.models as models
import torch.optim as optim
from transformers import get_linear_schedule_with_warmup

import math
from PIL import Image

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [ ]:
df_train = pd.read_csv('/kaggle/input/playing-card-value/train.csv')
df_train

,Filename,Total
0,000093681.jpg,37
1,000169578.jpg,50
2,000827954.jpg,35
3,001075659.jpg,45
4,001558425.jpg,53
...,...,...
4995,998695520.jpg,42
4996,998915603.jpg,34
4997,999615042.jpg,42
4998,999668496.jpg,46


In [ ]:

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Menggunakan VGG16 sebagai awal
        vgg16 = models.vgg16()
        self.features = vgg16.features

        # Menambahkan layer Conv2d tambahan
        self.conv1 = nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv3 = nn.Conv2d(128, 64, kernel_size=1, stride=1)

        # Inisialisasi bobot layer tambahan
        nn.init.xavier_uniform_(self.conv1.weight)
        nn.init.xavier_uniform_(self.conv2.weight)
        nn.init.xavier_uniform_(self.conv3.weight)

        # Menambahkan layer Linear
        self.linear1 = nn.Linear(64 * 7 * 7, 64)
        self.relu3 = nn.ReLU(inplace=True)
        self.linear2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.features(x)
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.conv3(x)
        x = x.view(x.size(0), -1)
        x = self.relu3(self.linear1(x))
        x = self.linear2(x)

        return x


class CustomDataset(Dataset):
    def __init__(self, dataframe, directory, transform=None):
        self.dataframe = dataframe
        self.directory = directory
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        image_path = self.directory + '/' + self.dataframe['Filename'].iloc[idx]
        image = Image.open(image_path).convert('RGB')
        label = self.dataframe['Total'].iloc[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
dataset = pd.read_csv('/kaggle/input/playing-card-value/train.csv')
image_directory = '/kaggle/input/playing-card-value/train/train'

image_transforms = transforms.Compose([
    transforms.Resize((224, 224), interpolation=Image.BILINEAR),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
custom_dataset = CustomDataset(dataset, image_directory, transform=image_transforms)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
train_size = int(0.8 * len(custom_dataset))
val_size = len(custom_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(custom_dataset, [train_size, val_size])

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
model = CNNModel()

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 51
total_steps = len(train_loader) * 20
warmup_steps = int(0.1 * total_steps)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
criterion = criterion.to(device)

for epoch in range(31, num_epochs):
    print(epoch, " Start")
    model.train()
    train_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch.unsqueeze(1).float())
        loss.backward()
        optimizer.step()
        scheduler.step()
        train_loss += loss.item() * X_batch.size(0)
    train_loss /= len(train_dataset)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch.unsqueeze(1).float())
            val_loss += loss.item() * X_batch.size(0)
    val_loss /= len(val_dataset)

    train_rmse = math.sqrt(train_loss)
    val_rmse = math.sqrt(val_loss)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train RMSE: {train_rmse:.4f}, Val Loss: {val_loss:.4f}, Val RMSE: {val_rmse:.4f}")

31  Start
Epoch [32/51], Train Loss: 5.3035, Train RMSE: 2.3029, Val Loss: 10.3849, Val RMSE: 3.2226
32  Start
Epoch [33/51], Train Loss: 25.9200, Train RMSE: 5.0912, Val Loss: 16.8524, Val RMSE: 4.1052
33  Start
Epoch [34/51], Train Loss: 12.1623, Train RMSE: 3.4874, Val Loss: 7.5196, Val RMSE: 2.7422
34  Start
Epoch [35/51], Train Loss: 5.7974, Train RMSE: 2.4078, Val Loss: 8.5607, Val RMSE: 2.9259
35  Start
Epoch [36/51], Train Loss: 2.8863, Train RMSE: 1.6989, Val Loss: 5.0722, Val RMSE: 2.2521
36  Start
Epoch [37/51], Train Loss: 2.6054, Train RMSE: 1.6141, Val Loss: 4.7733, Val RMSE: 2.1848
37  Start
Epoch [38/51], Train Loss: 2.1725, Train RMSE: 1.4739, Val Loss: 4.8808, Val RMSE: 2.2093
38  Start
Epoch [39/51], Train Loss: 1.5790, Train RMSE: 1.2566, Val Loss: 3.1934, Val RMSE: 1.7870
39  Start
Epoch [40/51], Train Loss: 0.7876, Train RMSE: 0.8874, Val Loss: 3.1655, Val RMSE: 1.7792
40  Start
Epoch [41/51], Train Loss: 0.6741, Train RMSE: 0.8210, Val Loss: 3.2052, Val RMSE: 1.7

In [ ]:
torch.save(model.state_dict(), '/kaggle/working/model_new8.pth')

In [ ]:

checkpoint = torch.load('/kaggle/working/model_new8.pth')
model = model.to(device)
model.load_state_dict(checkpoint)


<All keys matched successfully>

In [ ]:
model.named_parameters()

<generator object Module.named_parameters at 0x78e340581a80>

In [ ]:

image_path = "/kaggle/input/playing-card-value/test/test/003212838.jpg"
image_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
image = Image.open(image_path).convert('RGB')
image = image_transforms(image)

output = model(image.unsqueeze(0).to(device))
predicted_value = output.item()

prediction_data = {'Filename': image_path, 'PredictedValue': predicted_value}
prediction_data


{'Filename': '/kaggle/input/playing-card-value/test/test/003212838.jpg',
 'PredictedValue': 54.64946746826172}

In [ ]:

def predict_image_reg(path):
    global model
    image_path = dirname+'/'+filename
    image_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert('RGB')
    image = image_transforms(image)

    output = model(image.unsqueeze(0).to(device))
    predicted_value = output.item()

    return predicted_value


In [ ]:

pred_data = {
    'Filename':[],
    'Total':[]
}

for dirname, _, filenames in os.walk('/kaggle/input'):
    if dirname == '/kaggle/input/playing-card-value/test/test':
        filenames = sorted(filenames)
        for filename in filenames:

            curr_path = os.path.join(dirname, filename)
            pred_data['Filename'].append(filename)
            pred_data['Total'].append(predict_image_reg(curr_path))



In [ ]:

result_df = pd.DataFrame(pred_data)
result_df[['Filename', 'Total']].to_csv('result_image_new8.csv', index=False)
